In [4]:
import torch
import scipy.io as sio
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetClassifier
import torch.optim as optim
from CompactCNN import CompactCNN
import utils
import tensorboardX

Learning rate:
fix: batch = 100
epoch = 700 / (train/batch_size)
0.1 0.05 0.01 0.001

batch size:
fix: lr = 0.05
epoch = 700 / (train/batch_size)
50 - 250 - 500 1000

In [50]:
lr=0.001
batch_size = 50
n_epoch = int(700/(1796/batch_size))
path = "/home/longtd/Downloads/A-Compact-and-Interpretable-Convolutional-Neural-Network-for-Single-Channel-EEG-main/logging/"
# path = f"{path}/batch={batch_size}"
path = f"{path}/lr={lr}&batch_size={batch_size}"
utils.create_folders_if_necessary(path)
tb_writer = tensorboardX.SummaryWriter(path)

In [51]:
torch.cuda.empty_cache()
torch.manual_seed(0)
np.set_printoptions(2)

filename = r'dataset.mat' 
    
tmp = sio.loadmat(filename)
xdata=np.array(tmp['EEGsample'])
label=np.array(tmp['substate'])
subIdx=np.array(tmp['subindex'])

label.astype(int)
subIdx.astype(int)

samplenum=label.shape[0]


channelnum=30
subjnum=11
samplelength=3
sf=128

ydata=np.zeros(samplenum,dtype=np.longlong)

for i in range(samplenum):
    ydata[i]=label[i]

selectedchan=[28]

xdata=xdata[:,selectedchan,:]
channelnum=len(selectedchan)

results=np.zeros(subjnum)
for i in range(1,subjnum+1):

    culmulative = 0
    trainindx=np.where(subIdx != i)[0] 
    xtrain=xdata[trainindx]   
    x_train = xtrain.reshape(xtrain.shape[0],1,channelnum, samplelength*sf)
    y_train=ydata[trainindx]
                
    testindx=np.where(subIdx == i)[0]    
    xtest=xdata[testindx]
    x_test = xtest.reshape(xtest.shape[0], 1,channelnum, samplelength*sf)
    y_test=ydata[testindx]

    train = torch.utils.data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

    my_net = CompactCNN().double().cuda()

    optimizer = optim.Adam(my_net.parameters(), lr=lr)
    loss_class = torch.nn.NLLLoss().cuda()

    for p in my_net.parameters():
        p.requires_grad = True
      
    for epoch in range(n_epoch):
        for j, data in enumerate(train_loader, 0):
            header_t = [f"subject-{i} as test"]
            data_t = []

            inputs, labels = data                
            
            input_data = inputs.cuda()
            class_label = labels.cuda()

            my_net.zero_grad()
            my_net.train()          

            class_output= my_net(input_data) 
            err_s_label = loss_class(class_output, class_label)
            err = err_s_label 
            # print (err)
            data_t.append(err.tolist())
            err.backward()
            optimizer.step()
            for field, value in zip(header_t, data_t):
                tb_writer.add_scalar(field, value, culmulative)
                culmulative+=1


    my_net.train(False)
    with torch.no_grad():
        x_test =  torch.DoubleTensor(x_test).cuda()
        answer = my_net(x_test)
        probs=answer.cpu().numpy()
        preds       = probs.argmax(axis = -1)  
        acc=accuracy_score(y_test, preds)

        print("subject acc = %f" % acc)
        results[i-1]=acc
print('mean accuracy:',np.mean(results))

subject acc = 0.750000
subject acc = 0.522727
subject acc = 0.646667
subject acc = 0.790541
subject acc = 0.816964
subject acc = 0.807229
subject acc = 0.676471
subject acc = 0.674242
subject acc = 0.859873
subject acc = 0.814815
subject acc = 0.725664
mean accuracy: 0.7350174397166434


In [76]:
import time
def train_loop(lr, batch_size):
    n_epoch = int(400/(1796/batch_size))
    print('Start train for lr=%g, batch_size=%d, n_epoch=%d' % (lr, batch_size, n_epoch))
    start_time = time.time()
    torch.cuda.empty_cache()
    torch.manual_seed(0)
    np.set_printoptions(2)

    filename = r'dataset.mat' 
        
    tmp = sio.loadmat(filename)
    xdata=np.array(tmp['EEGsample'])
    label=np.array(tmp['substate'])
    subIdx=np.array(tmp['subindex'])

    label.astype(int)
    subIdx.astype(int)

    samplenum=label.shape[0]


    channelnum=30
    subjnum=11
    samplelength=3
    sf=128

    ydata=np.zeros(samplenum,dtype=np.longlong)

    for i in range(samplenum):
        ydata[i]=label[i]

    selectedchan=[28]

    xdata=xdata[:,selectedchan,:]
    channelnum=len(selectedchan)

    results=np.zeros(subjnum)
    for i in range(1,subjnum+1):

        culmulative = 0
        trainindx=np.where(subIdx != i)[0] 
        xtrain=xdata[trainindx]   
        x_train = xtrain.reshape(xtrain.shape[0],1,channelnum, samplelength*sf)
        y_train=ydata[trainindx]
                    
        testindx=np.where(subIdx == i)[0]    
        xtest=xdata[testindx]
        x_test = xtest.reshape(xtest.shape[0], 1,channelnum, samplelength*sf)
        y_test=ydata[testindx]

        train = torch.utils.data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

        my_net = CompactCNN().double().cuda()

        optimizer = optim.Adam(my_net.parameters(), lr=lr)
        loss_class = torch.nn.NLLLoss().cuda()

        for p in my_net.parameters():
            p.requires_grad = True
        
        for epoch in range(n_epoch):
            for j, data in enumerate(train_loader, 0):
                header_t = [f"subject-{i} as test"]
                data_t = []

                inputs, labels = data                
                
                input_data = inputs.cuda()
                class_label = labels.cuda()

                my_net.zero_grad()
                my_net.train()          

                class_output= my_net(input_data) 
                err_s_label = loss_class(class_output, class_label)
                err = err_s_label 
                # print (err)
                data_t.append(err.tolist())
                err.backward()
                optimizer.step()
                for field, value in zip(header_t, data_t):
                    tb_writer.add_scalar(field, value, culmulative)
                    culmulative+=1


        my_net.train(False)
        with torch.no_grad():
            x_test =  torch.DoubleTensor(x_test).cuda()
            answer = my_net(x_test)
            probs=answer.cpu().numpy()
            preds       = probs.argmax(axis = -1)  
            acc=accuracy_score(y_test, preds)
            results[i-1]=acc
    print("\tTrain time= %.3f seconds" % (time.time() - start_time))
    print('\tmean accuracy: %.4f' % np.mean(results))

In [77]:
lr_list = [0.1, 0.05, 0.01, 0.001]
batch_size_list = [50, 250, 500, 1000]

for lr_a in lr_list:
    for bz in batch_size_list:
        train_loop(lr_a, bz)

Start train for lr=0.1, batch_size=50, n_epoch=11
	Train time= 15.713 seconds
	mean accuracy: 0.7131
Start train for lr=0.1, batch_size=250, n_epoch=55
	Train time= 55.403 seconds
	mean accuracy: 0.7190
Start train for lr=0.1, batch_size=500, n_epoch=111
	Train time= 112.756 seconds
	mean accuracy: 0.7032
Start train for lr=0.1, batch_size=1000, n_epoch=222
	Train time= 229.354 seconds
	mean accuracy: 0.6892
Start train for lr=0.05, batch_size=50, n_epoch=11
	Train time= 16.440 seconds
	mean accuracy: 0.7221
Start train for lr=0.05, batch_size=250, n_epoch=55
	Train time= 56.791 seconds
	mean accuracy: 0.7147
Start train for lr=0.05, batch_size=500, n_epoch=111
	Train time= 112.849 seconds
	mean accuracy: 0.7143
Start train for lr=0.05, batch_size=1000, n_epoch=222
	Train time= 229.624 seconds
	mean accuracy: 0.7015
Start train for lr=0.01, batch_size=50, n_epoch=11
	Train time= 15.827 seconds
	mean accuracy: 0.7228
Start train for lr=0.01, batch_size=250, n_epoch=55
	Train time= 56.10